Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')


In [3]:
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
            
  
    #creating column year 
    X['year'] = pd.DatetimeIndex(X['date_recorded']).year
    

    #creating a new new column years between inspection
    X['construction_year'] = X['construction_year'].replace({0:2010})
    X['yrs_bw_insp'] = X['year'] - X['construction_year']  
    
    #to avoid error later,otherwise later when creating new features a positive will be divided by 0
    X['population'] = X['population'].replace({0:1}) 
    X['amount_tsh'] = X['amount_tsh'].replace({0:1})
    X['yrs_bw_insp'] = X['yrs_bw_insp'].replace({0:1})
    X['popu_per_year'] = X['population'] / X['yrs_bw_insp']
    X['water_per_person'] = X['amount_tsh']/X['population']

 


    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(columns=['quantity_group'])
    
    
    
    X= X.reset_index()

    # return the wrangled dataframe
    return X


train = wrangle(train)
test = wrangle(test)

In [4]:
# The status_group column is the target
target = 'status_group'

X_train = train.drop(columns=target)
y_train = train[target]
X_test = test

In [8]:
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(
    ce.ordinal.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=42)
)

param_distributions = {
   
    
#     Remove this hyperparameter, because of an issue: 
#     https://github.com/scikit-learn-contrib/categorical-encoding/issues/184
#     'targetencoder__smoothing': uniform(1, 1000), 
    
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5,10,15,20,25,None],
    'randomforestclassifier__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=7, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-1)]: Done   9 out of  21 | elapsed:  2.4min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done  12 out of  21 | elapsed:  2.7min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  15 out of  21 | elapsed:  3.1min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  18 out of  21 | elapsed:  4.4min remaining:   43.7s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  6.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  6.8min finished


In [9]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation Train Accuracy Score', -search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.26478111843703844, 'randomforestclassifier__n_estimators': 276, 'simpleimputer__strategy': 'mean'}
Cross-validation Train Accuracy Score -0.8048821548821549


In [10]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,2,1,5,4,6,0,3
mean_fit_time,118.363,72.3335,8.40436,139.207,242.026,38.5625,101.193
std_fit_time,3.53723,1.12891,0.237733,7.27912,2.00685,0.129122,5.06123
mean_score_time,2.97025,2.40922,1.06382,1.17885,1.99345,2.3793,1.09507
std_score_time,0.0218445,0.302118,0.0692525,0.0982504,0.0309582,0.0652092,0.0276514
param_randomforestclassifier__max_depth,20,15,25,20,15,10,5
param_randomforestclassifier__max_features,0.264781,0.25502,0.0497335,0.857497,0.832874,0.146027,0.907301
param_randomforestclassifier__n_estimators,276,207,50,115,347,223,209
param_simpleimputer__strategy,mean,median,mean,median,mean,mean,mean
params,"{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 25, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand..."
split0_test_score,0.803687,0.796414,0.798434,0.799444,0.795808,0.755707,0.723232


In [15]:
pipeline = search.best_estimator_

In [ ]:


X_test = test[features]


y_pred = pipeline.predict(X_test)
#mae = mean_absolute_error(y_test, y_pred)


###RandomizedSearchCV for XGBoost

In [9]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

pipeline = make_pipeline(
       #ce.OneHotEncoder(use_cat_names=True),
       ce.ordinal.OrdinalEncoder(),
       SimpleImputer(),
       StandardScaler(),
       xgb.XGBClassifier(random_state=42)
)
   
param_distributions = {
                'simpleimputer__strategy': ['mean', 'median'], 
    'xgbclassifier__n_estimators': randint(10, 30), 
    'xgbclassifier__max_depth': [5, 10, 12, 15, 20], 
    'xgbclassifier__colsample_bytree': [0.3,0.5,0.6,0.8,0.9], 
    'xgbclassifier__min_child_weight':[0.5,0.6,0.7]

} 

search = RandomizedSearchCV(
                             pipeline,
                            param_distributions=param_distributions,
                            n_iter=7,
                            cv= 3,
                            scoring='accuracy',
                            verbose=10,
                            return_train_score= True,
                            n_jobs=-1
 )   
    
    
    
search.fit(X_train,y_train);

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done   9 out of  21 | elapsed:   32.1s remaining:   42.8s
[Parallel(n_jobs=-1)]: Done  12 out of  21 | elapsed:   40.2s remaining:   30.1s
[Parallel(n_jobs=-1)]: Done  15 out of  21 | elapsed:   46.1s remaining:   18.4s
[Parallel(n_jobs=-1)]: Done  18 out of  21 | elapsed:   56.8s remaining:    9.4s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.1min finished


In [10]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation Train Accuracy Score', -search.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'median', 'xgbclassifier__colsample_bytree': 0.3, 'xgbclassifier__max_depth': 20, 'xgbclassifier__min_child_weight': 0.5, 'xgbclassifier__n_estimators': 23}
Cross-validation Train Accuracy Score -0.8029292929292929


In [11]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,0,2,6,1,4,5,3
mean_fit_time,23.3549,15.6531,19.6564,25.5078,13.4886,17.7429,10.0042
std_fit_time,0.313387,0.565292,1.08115,0.211459,0.901457,0.941707,0.377382
mean_score_time,1.38696,0.799861,0.550225,0.940817,0.629649,0.591418,0.515957
std_score_time,0.0835634,0.0214057,0.0834696,0.042628,0.0248396,0.0424623,0.0286057
param_simpleimputer__strategy,median,mean,mean,median,median,mean,median
param_xgbclassifier__colsample_bytree,0.3,0.3,0.5,0.6,0.5,0.5,0.6
param_xgbclassifier__max_depth,20,15,15,20,10,10,10
param_xgbclassifier__min_child_weight,0.5,0.7,0.6,0.7,0.5,0.6,0.7
param_xgbclassifier__n_estimators,23,19,23,14,16,26,10
params,"{'simpleimputer__strategy': 'median', 'xgbclas...","{'simpleimputer__strategy': 'mean', 'xgbclassi...","{'simpleimputer__strategy': 'mean', 'xgbclassi...","{'simpleimputer__strategy': 'median', 'xgbclas...","{'simpleimputer__strategy': 'median', 'xgbclas...","{'simpleimputer__strategy': 'mean', 'xgbclassi...","{'simpleimputer__strategy': 'median', 'xgbclas..."


###Stretch Goals

In [22]:
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
            
  
    #creating column year 
    X['year'] = pd.DatetimeIndex(X['date_recorded']).year
    

    #creating a new new column years between inspection
    X['yrs_bw_insp'] = X['year'] - X['construction_year']                   
    
    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(columns=['quantity_group'])
    
    #frequency encoding of imp features
    ae = X.groupby('lga').size()/len(X)
    X['lga'] = X['lga'].map(ae)

    be = X.groupby('waterpoint_type').size()/len(X)
    X['waterpoint_type'] = X['waterpoint_type'].map(ae)

    ce = X.groupby('extraction_type_class').size()/len(X)
    X['extraction_type_class'] = X['extraction_type_class'].map(ce)

    de =X.groupby('scheme_name').size()/len(X)
    X['scheme_name'] = X['scheme_name'].map(de)

    ee = X.groupby('funder').size()/len(X)
    X['funder'] = X['funder'].map(ee)

    fe =X.groupby('installer').size()/len(X)
    X['installer'] = X['installer'].map(fe)

    
    # return the wrangled dataframe
    return X


train = wrangle(train)
test = wrangle(test)

In [24]:
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(
    ce.ordinal.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=42)
)

param_distributions = {
   
    
#     Remove this hyperparameter, because of an issue: 
#     https://github.com/scikit-learn-contrib/categorical-encoding/issues/184
#     'targetencoder__smoothing': uniform(1, 1000), 
    
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5,10,15,20,25,None],
    'randomforestclassifier__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=7, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done   9 out of  21 | elapsed:  2.0min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  12 out of  21 | elapsed:  2.4min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  15 out of  21 | elapsed:  2.6min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  18 out of  21 | elapsed:  3.1min remaining:   30.9s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  4.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  4.1min finished


In [25]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation Train Accuracy Score', -search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.1851709075087814, 'randomforestclassifier__n_estimators': 57, 'simpleimputer__strategy': 'median'}
Cross-validation Train Accuracy Score -0.8043939393939394


In [26]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,5,3,4,6,2,0,1
mean_fit_time,15.052,59.3448,101.731,89.4277,12.2905,47.7716,146.39
std_fit_time,1.4007,0.875537,6.50808,3.08463,0.0818523,0.175541,1.60802
mean_score_time,0.863359,2.51328,1.57722,1.15923,0.699129,2.52709,1.65458
std_score_time,0.125875,0.0712582,0.201665,0.218383,0.0141269,0.0697186,0.0672049
param_randomforestclassifier__max_depth,20,None,None,15,10,10,5
param_randomforestclassifier__max_features,0.185171,0.19113,0.429138,0.720622,0.217889,0.174994,0.807342
param_randomforestclassifier__n_estimators,57,206,183,208,60,336,427
param_simpleimputer__strategy,median,median,median,mean,median,median,median
params,"{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand..."
split0_test_score,0.80798,0.807525,0.804899,0.795707,0.757929,0.756364,0.716566


In [8]:
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

pipeline = make_pipeline(
       #ce.OneHotEncoder(use_cat_names=True),
       ce.ordinal.OrdinalEncoder(),
       SimpleImputer(),
       StandardScaler(),
       xgb.XGBClassifier(random_state=42)
)
   
param_distributions = {
                'simpleimputer__strategy': ['mean', 'median'], 
    'xgbclassifier__n_estimators': randint(10, 30), 
    'xgbclassifier__max_depth': [5, 10, 12, 15, 20], 
    'xgbclassifier__colsample_bytree': [0.3,0.5,0.6,0.8,0.9], 
    'xgbclassifier__min_child_weight':[0.5,0.6,0.7]

} 

search = RandomizedSearchCV(
                             pipeline,
                            param_distributions=param_distributions,
                            n_iter=7,
                            cv= 3,
                            scoring='accuracy',
                            verbose=10,
                            return_train_score= True,
                            n_jobs=-1
 )   
    
    
    
search.fit(X_train,y_train);

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done   9 out of  21 | elapsed:   57.8s remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  12 out of  21 | elapsed:  1.1min remaining:   49.4s
[Parallel(n_jobs=-1)]: Done  15 out of  21 | elapsed:  1.4min remaining:   32.6s
[Parallel(n_jobs=-1)]: Done  18 out of  21 | elapsed:  1.5min remaining:   14.5s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.7min finished


In [12]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation Train Accuracy Score', -search.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'median', 'xgbclassifier__colsample_bytree': 0.3, 'xgbclassifier__max_depth': 20, 'xgbclassifier__min_child_weight': 0.5, 'xgbclassifier__n_estimators': 23}
Cross-validation Train Accuracy Score -0.8029292929292929


In [13]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,0,2,6,1,4,5,3
mean_fit_time,23.3549,15.6531,19.6564,25.5078,13.4886,17.7429,10.0042
std_fit_time,0.313387,0.565292,1.08115,0.211459,0.901457,0.941707,0.377382
mean_score_time,1.38696,0.799861,0.550225,0.940817,0.629649,0.591418,0.515957
std_score_time,0.0835634,0.0214057,0.0834696,0.042628,0.0248396,0.0424623,0.0286057
param_simpleimputer__strategy,median,mean,mean,median,median,mean,median
param_xgbclassifier__colsample_bytree,0.3,0.3,0.5,0.6,0.5,0.5,0.6
param_xgbclassifier__max_depth,20,15,15,20,10,10,10
param_xgbclassifier__min_child_weight,0.5,0.7,0.6,0.7,0.5,0.6,0.7
param_xgbclassifier__n_estimators,23,19,23,14,16,26,10
params,"{'simpleimputer__strategy': 'median', 'xgbclas...","{'simpleimputer__strategy': 'mean', 'xgbclassi...","{'simpleimputer__strategy': 'mean', 'xgbclassi...","{'simpleimputer__strategy': 'median', 'xgbclas...","{'simpleimputer__strategy': 'median', 'xgbclas...","{'simpleimputer__strategy': 'mean', 'xgbclassi...","{'simpleimputer__strategy': 'median', 'xgbclas..."
